# Intro

In [1]:
import spacy

%load_ext nb_black

nlp = spacy.load("en_core_web_sm")

<IPython.core.display.Javascript object>

In [2]:
# Process sentences 'Hello, world. Antonio is learning Python.' using spaCy
doc = nlp(u"Hello, world. Antonio is learning Python.")

<IPython.core.display.Javascript object>

## Get tokens and sentences

#### What is a Token?
A token is a single chopped up element of the sentence, which could be a word or a group of words to analyse. The task of chopping the sentence up is called "tokenisation".

Example: The following sentence can be tokenised by splitting up the sentence into individual words.

	"Antonio is learning Python!"
	["Antonio","is","learning","Python!"]

In [3]:
# Get first token of the processed document
token = doc[0]
print(token)

# Print sentences (one sentence per line)
for sent in doc.sents:
    print(sent)

Hello
Hello, world.
Antonio is learning Python.


<IPython.core.display.Javascript object>

## Part of speech tags

#### What is a Speech Tag?
A speech tag is a context sensitive description of what a word means in the context of the whole sentence.
More information about the kinds of speech tags which are used in NLP can be [found here](http://www.winwaed.com/blog/2011/11/08/part-of-speech-tags/).

Examples:

1. CARDINAL, Cardinal Number - 1,2,3
2. PROPN, Proper Noun, Singular - "Jan", "Javier", "Antonio", "Italy"
3. INTJ, Interjection - "Ohhhhhhhhhhh"

In [4]:
# For each token, print corresponding part of speech tag

for token in doc:
    print(token, '\t\t', token.pos_)


Hello 		 INTJ
, 		 PUNCT
world 		 NOUN
. 		 PUNCT
Antonio 		 PROPN
is 		 AUX
learning 		 VERB
Python 		 PROPN
. 		 PUNCT


<IPython.core.display.Javascript object>

In [5]:
from spacy import displacy

<IPython.core.display.Javascript object>

In [6]:
displacy.serve(doc, style='dep')



/home/alessio/anaconda3/envs/deep_learning/lib/python3.9/site-packages/spacy/displacy/__init__.py:97: UserWarning: [W011] It looks like you're calling displacy.serve from within a Jupyter notebook or a similar environment. This likely means you're already running a local web server, so there's no need to make displaCy start another one. Instead, you should be able to replace displacy.serve with displacy.render to show the visualization.
  warnings.warn(Warnings.W011)



Using the 'dep' visualizer
Serving on http://0.0.0.0:5000 ...

Shutting down server on port 5000.


<IPython.core.display.Javascript object>

In [7]:
displacy.render(doc, style = "ent",jupyter = True)


<IPython.core.display.Javascript object>

We have said that dependency structures are represented by directed graphs that satisfy the following constraints:

1. There is a single designated root node that has no incoming arcs.

2. With the exception of the root node, each vertex has exactly one incoming arc.

3. There is a unique path from the root node to each vertex in V.

You can inspect the head of each token by invoking the `.head` attribute of a spaCy token:


In [8]:
doc[2]

world

<IPython.core.display.Javascript object>

In [9]:
doc[2].head

Hello

<IPython.core.display.Javascript object>

So how would you search for the root?

Since there is a unique path from the root node to each vertex in V, there's only one root node that has no incoming arcs, we can search for the token which have as head itself!

In [10]:
for token in doc:
    if token.head == token:
        print(token)

Hello
learning


<IPython.core.display.Javascript object>

As expected, since there were two sentences in the doc, we got two roots.

We can also build a function that, given a spaCy token, gives the path till the root:

In [11]:
# Define a function to find the path to the root of each word in a sentence
def path_to_the_root(token):
    if token.head == token:
        return token
    else:
        print(f'{token}->{token.head}')
        path_to_the_root(token.head)

<IPython.core.display.Javascript object>

In [12]:
head = path_to_the_root(doc[4])

Antonio->learning


<IPython.core.display.Javascript object>

In [13]:
print(head)

None


<IPython.core.display.Javascript object>

## Embeddings 

An embedding is a fixed sizes numerical vector that attempts to encode some semantic meaning of the word or sentence it is encoding. The distributional hypothesis is usually the concept behind most embeddings. This hypothesis states that words which often have the same neighboring words tend to be semantically similar. For example if 'football' and 'basketball' usually appear close the word 'play' we assume that they will be semantically similar. An algorithm that is based on this concept is Word2Vec. A common way of obtaining sentence embeddings is to average the word embeddings inside the sentence and use that average as the representation of the whole sentence. 

- In spacy every token has its embedding.
- It is under the attribute 'vector'.
- In spacy embeddings are of size 96 or 128.


Obtain the embeddings of all the tokens.

In [14]:
for token in doc:
    print(token.vector)

[-0.44524243 -0.00310461  0.5343243  -0.17235672 -0.6761433  -0.38657025
 -0.3381957  -0.02988945 -0.00555246 -0.84159493  0.16327694 -0.57697415
 -0.68480617  0.00622424  0.07834896 -0.30333096  1.5165389  -0.59602
  0.03028199 -0.43093356  0.03739309 -0.8697863  -0.8442886   0.0344111
  0.25991035 -0.22334436  0.09180419 -0.40757135 -0.36183017 -0.38972035
  0.46326637 -0.54095584  0.25360817 -0.6503706  -0.23880535  0.45702097
  0.7022171   0.42939293  0.7362815   0.30060843 -0.6067468  -0.40308905
  0.24224597  1.1391797  -0.31396425 -0.52889717  0.06228442  1.250154
 -0.5838891   0.99806213 -0.65224385 -0.04109746  0.94972086 -0.5786103
 -0.95549035 -0.96690434  0.44947106  0.59939575 -0.04105932 -0.14472458
  0.146662   -0.524665    1.1935519   0.18990526  0.19144967 -0.3871125
 -0.6585389  -0.3067224   0.68035066 -0.5279468   1.1744324   0.18890297
 -0.17509712 -0.12649429  0.2876051   0.47623295 -0.9217448  -0.12227535
 -0.7543309  -0.08898088 -0.55470145 -0.36096507 -0.3211115

<IPython.core.display.Javascript object>

## Semantic similarity 

To compute the semantic similarity between two sentences, $u$ and $v$, we measure the cossine similarity between the two sentence embeddings. The formula is as follows:

$sim(u, v) = \frac{u \cdot v}{||u|| ||v||} $


Use the following formula to get the semantic similarity betwen the words in doc.
Feel free to test it between differente words too

In [15]:
import numpy as np

<IPython.core.display.Javascript object>

In [16]:
def semantic_sim(u,v):
    return (u @ v) / (np.sqrt(sum(u**2)) * np.sqrt(sum(v**2)))

<IPython.core.display.Javascript object>

In [17]:
print(semantic_sim(doc[1].vector, doc[3].vector))

0.17669735320098082


<IPython.core.display.Javascript object>

# Pride and Prejudice analysis

We would like to:

- Extract the names of all the characters from the book (e.g. Elizabeth, Darcy, Bingley)
- Visualize characters' occurences with regards to relative position in the book
- Authomatically describe any character from the book
- Find out which characters have been mentioned in a context of marriage
- Build keywords extraction that could be used to display a word cloud (example)

To load the text file, it is convinient to decode using the utf-8 standard:

In [18]:
def read_file(file_name):
    with open(file_name, "r", encoding="utf-8") as file:
        return file.read()

<IPython.core.display.Javascript object>

### Process full text

In [32]:
text = read_file("data/pride_and_prejudice.txt")
# Process the text

doc = nlp(text)

<IPython.core.display.Javascript object>

In [34]:
dir(doc.sents)

['__class__',
 '__del__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__lt__',
 '__name__',
 '__ne__',
 '__new__',
 '__next__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'close',
 'gi_code',
 'gi_running',
 'gi_yieldfrom',
 'send',
 'throw']

<IPython.core.display.Javascript object>

In [35]:
print(doc.sents)

<built-in method __sizeof__ of generator object at 0x7f8fadcf8ca0>


<IPython.core.display.Javascript object>

In [31]:
# How many sentences are in the book (Pride & Prejudice)?

count = 0
for sent in sents:
    count +=1

print(count)

# Print sentences from index 10 to index 15, to make sure that we have parsed the correct book

sents
    


0


<IPython.core.display.Javascript object>

## Find all the personal names

[Hint](# "List doc.ents and check ent.label_")

In [ ]:
# Extract all the personal names from Pride & Prejudice and count their occurrences.
# Expected output is a list in the following form: [('elizabeth', 622), ('darcy', 312), ('jane', 286), ('bennet', 266) ...].

from collections import Counter, defaultdict


def find_character_occurences(doc):
    """
    Return a list of actors from `doc` with corresponding occurences.

    :param doc: Spacy NLP parsed document
    :return: list of tuples in form
        [('elizabeth', 622), ('darcy', 312), ('jane', 286), ('bennet', 266)]
    """

    characters = Counter()
    # your code here


print(find_character_occurences(processed_text)[:20])

## Plot characters personal names as a time series 

In [ ]:
# Matplotlib Jupyter HACK
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

We can investigate where a particular entity occurs in the text. We can do it just accessing the `.start` attribute of an entity:

[Hint](# "ent.start")

In [ ]:
# List all the start positions of person entities

So we can create a function that stores all the offsets of every character:
   
   
[Hint](# "Create a dictionary with the lowered lemmas [ent.lemma_.lower()] and associate a list of all the ent.starts")

In [ ]:
# Plot characters' mentions as a time series relative to the position of the actor's occurrence in a book.

def get_character_offsets(doc):
    """
    For every character in a `doc` collect all the occurences offsets and store them into a list. 
    The function returns a dictionary that has actor lemma as a key and list of occurences as a value for every character.
    
    :param doc: Spacy NLP parsed document
    :return: dict object in form
        {'elizabeth': [123, 543, 4534], 'darcy': [205, 2111]}
    """
            
    return dict(character_offsets)

character_occurences = get_character_offsets(processed_text)

In [ ]:
character_occurences

[Hint](# "Use the character offsets for each character as x")

In [ ]:
# Plot the histogram of the character occurrences in the whole text
NUM_BINS = 20

def plot_character_hist(character_offsets, character_label, cumulative=False):
    pass

In [ ]:
plot_character_hist(character_occurences, "elizabeth")

In [ ]:
plot_character_hist(character_occurences, "darcy")

### Cumulative occurrences

In [ ]:
plot_character_hist(character_occurences, "elizabeth", cumulative=True)

In [ ]:
plot_character_hist(character_occurences, "darcy", cumulative=True)

### Spacy parse tree in action

[Hint](# "ent.subtree, token.pos_ == 'ADJ'") 

In [ ]:
# Find words (adjectives) that describe Mr. Darcy.

def get_character_adjectives(doc, character_lemma):
    """
    Find all the adjectives related to `character_lemma` in `doc`
    
    :param doc: Spacy NLP parsed document
    :param character_lemma: string object
    :return: list of adjectives related to `character_lemma`
    """
    
    adjectives = []
    for ent in processed_text.ents:
        # your code here
        pass
    
     for ent in processed_text.ents:
        if ent.lemma_.lower() == character_lemma:
            if ent.root.dep_ == 'nsubj':
                for child in ent.root.head.children:
                    if child.dep_ == 'acomp':
                        adjectives.append(child.lemma_)
                        
    return adjectives

print(get_character_adjectives(processed_text, 'darcy'))

In [ ]:
# Find words (adjectives) that describe Elizabeth.


print(get_character_adjectives(processed_text, 'elizabeth'))

For all the dependencies manual: https://nlp.stanford.edu/software/dependencies_manual.pdf

`acomp`: adjectival complement
*i.e.* an adjectival phrase which functions as the complement (like an object of the verb) e.g. "She looks very beautiful": *beautiful* is an adjectival complement of *looks*

`nsubj`: nominal subject
*i.e.* a noun phrase which is the syntactic subject of a clause. The head of this relation
might not always be a verb: when the verb is a copular verb, the root of the clause is the complement of
the copular verb, which can be an adjective or noun.
*e.g.* "Clinton defeated Dole". The relationship is *nsubj(defeated, Clinton)*

"The baby is cute". The relationship is *nsubj(cute, baby)*.

In the code, `.dep_`stands for syntactic dependency, *i.e.* the relation between tokens.

In [ ]:
processed_text.ents[30].root.dep_

[Hint](# "ent.label_, ent.root.head.lemma_") 

In [ ]:
# Find characters that are 'talking', 'saying', 'doing' the most. Find the relationship between 
# entities and corresponding root verbs.

character_verb_counter = Counter()


for ent in processed_text.ents:
    if # your code here:
        character_verb_counter[ent.text] += 1

print(character_verb_counter.most_common(10)) 

# do the same for talking and doing

print(character_verb_counter.most_common(10)) 


[Hint](# "ent.label_, ent.root.head.pos_") 

In [ ]:
# Find 20 most used verbs
verb_counter = Counter()

# your code here

print(verb_counter.most_common(20))

In [ ]:
# Create a dataframe with the most used verb and how many time a character used the verb

import pandas as pd
verb_characters = {}
verb_list = [verb[0] for verb in verb_counter.most_common(20)]
for ent in processed_text.ents:
    if ent.label_ == 'PERSON' and ent.root.head.lemma_ in verb_list:
        # complete the code
        pass


In [ ]:
df = pd.DataFrame(verb_characters).transpose().fillna(0)
df

In [ ]:
# drop the less meaningful columns
df = df[df.columns[df.sum()>=10]].sort_index()
df

In [ ]:
import seaborn as sns
%matplotlib inline
sns.heatmap(df, annot=True, cmap='Blues')
df.style.background_gradient(cmap='Blues')
